In [18]:
import requests
import geopandas as gpd
import pandas as pd
import numpy as np
import datetime
import json
import os


In [28]:
os.chdir('/Users/tomweatherburn/Library/CloudStorage/OneDrive-Personal/dev/tdubolyou.github.io/crime')
dir = os.getcwd()
print(dir)

/Users/tomweatherburn/Library/CloudStorage/OneDrive-Personal/dev/tdubolyou.github.io/crime


In [30]:
# Dictionary of dataset names and their respective URLs
datasets = {
    "mci": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Major_Crime_Indicators_Open_Data/FeatureServer/0/query",
    "hom": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Homicides_Open_Data_ASR_RC_TBL_002/FeatureServer/0/query",
    "gun": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Shooting_and_Firearm_Discharges_Open_Data/FeatureServer/0/query",
    "bike": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Bicycle_Thefts_Open_Data/FeatureServer/0/query",
    "car": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Theft_From_Motor_Vehicle_Open_Data/FeatureServer/0/query",
    "nbr": "https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/Neighbourhood_Crime_Rates_Open_Data/FeatureServer/0/query"
}

# Initialize a dictionary to store the GeoDataFrames
gdfs = {}

for dataset_name, url in datasets.items():
    all_features = []
    offset = 0
    while True:
        # Define request parameters
        params = {
            "where": "1=1",
            "outFields": "*",
            "outSR": "4326",
            "f": "geojson",
            "resultRecordCount": 2000,
            "resultOffset": offset
        }
        
        # Make the request
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            features = data.get("features", [])
            if not features:
                break  # Exit loop if no more features to fetch
            all_features.extend(features)
            offset += len(features)
        else:
            print(f"Failed to fetch data for {dataset_name}. Status code: {response.status_code}")
            break

    # # Get the current date
    # current_date = datetime.datetime.now()

    # # Get the current year
    # current_year = current_date.year

    # # Determine the current quarter
    # current_month = current_date.month
    # if 1 <= current_month <= 3:
    #     current_quarter = "Q1"
    # elif 4 <= current_month <= 6:
    #     current_quarter = "Q2"
    # elif 7 <= current_month <= 9:
    #     current_quarter = "Q3"
    # else:
    #     current_quarter = "Q4"
    # # Save the GeoDataFrame to a GeoJSON file
    # filename = f"data/{dataset_name}_{current_quarter}{current_year}.geojson"
    # gdf.to_file(filename, driver='GeoJSON')
    
    # Convert all_features to a GeoDataFrame directly without saving to a file
    gdfs[dataset_name] = gpd.GeoDataFrame.from_features(all_features)

# Now you have each GeoDataFrame stored in gdfs dictionary, accessible by its dataset name
# To access individual GeoDataFrames, you can directly use the dictionary:
gdf_mci = gdfs['mci']
gdf_hom = gdfs['hom']
gdf_gun = gdfs['gun']
gdf_bike = gdfs['bike']
gdf_car = gdfs['car']
gdf_nbr = gdfs['nbr']


In [31]:
print(gdf_mci.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 384687 entries, 0 to 384686
Data columns (total 30 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   geometry           384687 non-null  geometry
 1   OBJECTID           384687 non-null  int64   
 2   EVENT_UNIQUE_ID    384687 non-null  object  
 3   REPORT_DATE        384687 non-null  int64   
 4   OCC_DATE           384687 non-null  int64   
 5   REPORT_YEAR        384687 non-null  int64   
 6   REPORT_MONTH       384687 non-null  object  
 7   REPORT_DAY         384687 non-null  int64   
 8   REPORT_DOY         384687 non-null  int64   
 9   REPORT_DOW         384687 non-null  object  
 10  REPORT_HOUR        384687 non-null  int64   
 11  OCC_YEAR           384566 non-null  float64 
 12  OCC_MONTH          384566 non-null  object  
 13  OCC_DAY            384566 non-null  float64 
 14  OCC_DOY            384566 non-null  float64 
 15  OCC_DOW            384566 

In [78]:
# gdf_mci = gpd.read_file(dir+"/data/"+'mci.geojson')
# gdf_hom = gpd.read_file(dir+"/data/"+'hom.geojson')
# gdf_gun = gpd.read_file(dir+"/data/"+'gun.geojson')
# gdf_bike = gpd.read_file(dir+"/data/"+'bike.geojson')
# gdf_car = gpd.read_file(dir+"/data/"+'car.geojson')
# gdf_nbr = gpd.read_file(dir+"/data/"+'nbr.geojson')

In [32]:
#Filter down Redundant GDFS
gdf_gun = gdf_gun[gdf_gun['DEATH'] == 0] #Remove shooting deaths as they are redundant with homicides
gdf_car = gdf_car[gdf_car['MCI_CATEGORY'] == 'NonMCI']

In [33]:
##Add a crime field 
# Example for the Homicide GDF
gdf_hom['CRIME'] = 'Homicide'

# Example for the Non-Fatal Shootings GDF, assuming 'INJURIES' indicates non-fatal shootings
gdf_gun['CRIME'] = 'Shooting (Non-Fatal)'

# Example for the Bike Theft GDF
gdf_bike['CRIME'] = 'Bike Theft'

# For the MCI GDF, assuming 'MCI_CATEGORY' is present and you want to exclude 'NonMCI' values
gdf_mci['CRIME'] = gdf_mci['MCI_CATEGORY']

gdf_car['CRIME'] = 'Theft From Motor Vehicle'

# If there's a specific condition for other datasets, follow a similar pattern:
# gdf_x['CRIME'] = [appropriate logic here]


/Users/tomweatherburn/anaconda3/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/tomweatherburn/anaconda3/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [34]:
data = [gdf_mci, gdf_hom, gdf_gun, gdf_bike, gdf_car]
for gdf in data:

    print(gdf.columns)
    print(len(gdf))
    print(gdf['CRIME'].value_counts())

Index(['geometry', 'OBJECTID', 'EVENT_UNIQUE_ID', 'REPORT_DATE', 'OCC_DATE',
       'REPORT_YEAR', 'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW',
       'REPORT_HOUR', 'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW',
       'OCC_HOUR', 'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE',
       'UCR_EXT', 'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158',
       'HOOD_140', 'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84', 'CRIME'],
      dtype='object')
384687
CRIME
Assault            203679
Break and Enter     72144
Auto Theft          61216
Robbery             34756
Theft Over          12892
Name: count, dtype: int64
Index(['geometry', 'OBJECTID', 'EVENT_UNIQUE_ID', 'OCC_DATE', 'OCC_YEAR',
       'OCC_MONTH', 'OCC_DAY', 'OCC_DOW', 'OCC_DOY', 'DIVISION',
       'HOMICIDE_TYPE', 'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140',
       'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84', 'CRIME'],
      dtype='object')
1414
CRIME
Homicide    1414
Name: count, dtyp

In [35]:
# Convert columns Index to a list and print
print(list(gdf_nbr.columns))


['geometry', 'OBJECTID_1', 'AREA_NAME', 'HOOD_ID', 'ASSAULT_2014', 'ASSAULT_2015', 'ASSAULT_2016', 'ASSAULT_2017', 'ASSAULT_2018', 'ASSAULT_2019', 'ASSAULT_2020', 'ASSAULT_2021', 'ASSAULT_2022', 'ASSAULT_2023', 'ASSAULT_RATE_2014', 'ASSAULT_RATE_2015', 'ASSAULT_RATE_2016', 'ASSAULT_RATE_2017', 'ASSAULT_RATE_2018', 'ASSAULT_RATE_2019', 'ASSAULT_RATE_2020', 'ASSAULT_RATE_2021', 'ASSAULT_RATE_2022', 'ASSAULT_RATE_2023', 'AUTOTHEFT_2014', 'AUTOTHEFT_2015', 'AUTOTHEFT_2016', 'AUTOTHEFT_2017', 'AUTOTHEFT_2018', 'AUTOTHEFT_2019', 'AUTOTHEFT_2020', 'AUTOTHEFT_2021', 'AUTOTHEFT_2022', 'AUTOTHEFT_2023', 'AUTOTHEFT_RATE_2014', 'AUTOTHEFT_RATE_2015', 'AUTOTHEFT_RATE_2016', 'AUTOTHEFT_RATE_2017', 'AUTOTHEFT_RATE_2018', 'AUTOTHEFT_RATE_2019', 'AUTOTHEFT_RATE_2020', 'AUTOTHEFT_RATE_2021', 'AUTOTHEFT_RATE_2022', 'AUTOTHEFT_RATE_2023', 'BIKETHEFT_2014', 'BIKETHEFT_2015', 'BIKETHEFT_2016', 'BIKETHEFT_2017', 'BIKETHEFT_2018', 'BIKETHEFT_2019', 'BIKETHEFT_2020', 'BIKETHEFT_2021', 'BIKETHEFT_2022', 'BIKETH

In [36]:
# Common columns identified (as an example)
common_columns = ['EVENT_UNIQUE_ID', 'OCC_DATE', 'OCC_YEAR', 'OCC_DOW','DIVISION','HOOD_158','NEIGHBOURHOOD_158', 'CRIME','geometry']

# Additional necessary columns identified (example based on your provided columns)
additional_columns = ['OCC_HOUR','HOMICIDE_TYPE', 'INJURIES', 'BIKE_MAKE', 'BIKE_COST', 'STATUS', 'OFFENCE', 'PRIMARY_OFFENCE','LOCATION_TYPE', 'PREMISES_TYPE',]

In [37]:
for gdf in data:
    for col in common_columns:
        print(col in gdf_hom.columns)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [38]:
# Standardize GeoDataFrames
gdfs = [gdf_mci, gdf_hom, gdf_gun, gdf_bike, gdf_car]
for gdf in gdfs:
    for col in additional_columns:
        if col not in gdf.columns:
            gdf[col] = None  # Fill missing additional columns with None or an appropriate value

for gdf in gdfs:

    print(gdf.columns)
    print(len(gdf))



Index(['geometry', 'OBJECTID', 'EVENT_UNIQUE_ID', 'REPORT_DATE', 'OCC_DATE',
       'REPORT_YEAR', 'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW',
       'REPORT_HOUR', 'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW',
       'OCC_HOUR', 'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE',
       'UCR_EXT', 'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158',
       'HOOD_140', 'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84', 'CRIME',
       'HOMICIDE_TYPE', 'INJURIES', 'BIKE_MAKE', 'BIKE_COST', 'STATUS',
       'PRIMARY_OFFENCE'],
      dtype='object')
384687
Index(['geometry', 'OBJECTID', 'EVENT_UNIQUE_ID', 'OCC_DATE', 'OCC_YEAR',
       'OCC_MONTH', 'OCC_DAY', 'OCC_DOW', 'OCC_DOY', 'DIVISION',
       'HOMICIDE_TYPE', 'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140',
       'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84', 'CRIME', 'OCC_HOUR',
       'INJURIES', 'BIKE_MAKE', 'BIKE_COST', 'STATUS', 'OFFENCE',
       'PRIMARY_OFFENCE', 'LOCATION_TYPE', 'PREMISES_T

In [39]:
# Select columns for each GDF before concatenating
gdfs = [gdf[common_columns + additional_columns] for gdf in gdfs if set(common_columns).issubset(gdf.columns)]

# Merge standardized GeoDataFrames
merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

# Save merged GeoDataFrame to a new GeoJSON file
#merged_gdf.to_file(dir+"/data/crime.geojson", driver='GeoJSON')

In [40]:
print(merged_gdf.info())
print(gdf_nbr.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 515948 entries, 0 to 515947
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   EVENT_UNIQUE_ID    515948 non-null  object  
 1   OCC_DATE           515948 non-null  int64   
 2   OCC_YEAR           515808 non-null  float64 
 3   OCC_DOW            515808 non-null  object  
 4   DIVISION           515948 non-null  object  
 5   HOOD_158           515948 non-null  object  
 6   NEIGHBOURHOOD_158  515948 non-null  object  
 7   CRIME              515948 non-null  object  
 8   geometry           515948 non-null  geometry
 9   OCC_HOUR           514534 non-null  object  
 10  HOMICIDE_TYPE      1414 non-null    object  
 11  INJURIES           5463 non-null    object  
 12  BIKE_MAKE          35325 non-null   object  
 13  BIKE_COST          32926 non-null   float64 
 14  STATUS             35325 non-null   object  
 15  OFFENCE            473746 

In [41]:
print(merged_gdf['CRIME'].value_counts())

for column in merged_gdf.columns:
    print(column)
print(gdf_nbr.columns)

CRIME
Assault                     203679
Theft From Motor Vehicle     89059
Break and Enter              72144
Auto Theft                   61216
Bike Theft                   35325
Robbery                      34756
Theft Over                   12892
Shooting (Non-Fatal)          5463
Homicide                      1414
Name: count, dtype: int64
EVENT_UNIQUE_ID
OCC_DATE
OCC_YEAR
OCC_DOW
DIVISION
HOOD_158
NEIGHBOURHOOD_158
CRIME
geometry
OCC_HOUR
HOMICIDE_TYPE
INJURIES
BIKE_MAKE
BIKE_COST
STATUS
OFFENCE
PRIMARY_OFFENCE
LOCATION_TYPE
PREMISES_TYPE
Index(['geometry', 'OBJECTID_1', 'AREA_NAME', 'HOOD_ID', 'ASSAULT_2014',
       'ASSAULT_2015', 'ASSAULT_2016', 'ASSAULT_2017', 'ASSAULT_2018',
       'ASSAULT_2019',
       ...
       'THEFTOVER_RATE_2017', 'THEFTOVER_RATE_2018', 'THEFTOVER_RATE_2019',
       'THEFTOVER_RATE_2020', 'THEFTOVER_RATE_2021', 'THEFTOVER_RATE_2022',
       'THEFTOVER_RATE_2023', 'POPULATION_2023', 'Shape__Area',
       'Shape__Length'],
      dtype='object', length=1

In [42]:
####Add HOOD_ID to Merged_gdf and create individual Neighbourhood point files

# Example: If the CRS is not set, explicitly set it
if merged_gdf.crs is None:
    merged_gdf.crs = gdf_nbr.crs

if 'index_right' in merged_gdf.columns:
    merged_gdf.drop(columns=['index_right'], inplace=True)

# Similarly, if you want to remove 'index_left' as well
if 'index_left' in merged_gdf.columns:
    merged_gdf.drop(columns=['index_left'], inplace=True)

# Perform a spatial join
merged_gdf = gpd.sjoin(merged_gdf, gdf_nbr[['HOOD_ID', 'geometry']], how='left', op='within')

print(merged_gdf.columns)

##Create individual Neighbourhood geojsons
for neighborhood_id in merged_gdf['HOOD_ID'].unique():
    # Filter the GeoDataFrame for the current neighborhood ID
    gdf_subset = merged_gdf[merged_gdf['HOOD_ID'] == neighborhood_id]
    
    # Define the filename using the neighborhood ID
    filename = f'/data/crime_{neighborhood_id}.geojson'
    
    # Save the subset GeoDataFrame to a GeoJSON file
    gdf_subset.to_file(dir+filename, driver='GeoJSON')
    
    print(f'Saved: {filename}')

/Users/tomweatherburn/anaconda3/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Index(['EVENT_UNIQUE_ID', 'OCC_DATE', 'OCC_YEAR', 'OCC_DOW', 'DIVISION',
       'HOOD_158', 'NEIGHBOURHOOD_158', 'CRIME', 'geometry', 'OCC_HOUR',
       'HOMICIDE_TYPE', 'INJURIES', 'BIKE_MAKE', 'BIKE_COST', 'STATUS',
       'OFFENCE', 'PRIMARY_OFFENCE', 'LOCATION_TYPE', 'PREMISES_TYPE',
       'index_right', 'HOOD_ID'],
      dtype='object')
Saved: /data/crime_84.0.geojson
Saved: /data/crime_78.0.geojson
Saved: /data/crime_163.0.geojson
Saved: /data/crime_28.0.geojson
Saved: /data/crime_151.0.geojson
Saved: /data/crime_61.0.geojson
Saved: /data/crime_150.0.geojson
Saved: /data/crime_100.0.geojson
Saved: /data/crime_55.0.geojson
Saved: /data/crime_94.0.geojson
Saved: /data/crime_164.0.geojson
Saved: /data/crime_91.0.geojson
Saved: /data/crime_143.0.geojson
Saved: /data/crime_120.0.geojson
Saved: /data/crime_81.0.geojson
Saved: /data/crime_136.0.geojson
Saved: /data/crime_73.0.geojson
Saved: /data/crime_37.0.geojson
Saved: /data/crime_123.0.geojson
Saved: /data/crime_160.0.geojson
Saved

/Users/tomweatherburn/anaconda3/envs/geo/lib/python3.10/site-packages/geopandas/io/file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


Saved: /data/crime_108.0.geojson
Saved: /data/crime_83.0.geojson
Saved: /data/crime_18.0.geojson
Saved: /data/crime_32.0.geojson
Saved: /data/crime_42.0.geojson
Saved: /data/crime_44.0.geojson
Saved: /data/crime_138.0.geojson
Saved: /data/crime_87.0.geojson
Saved: /data/crime_62.0.geojson
Saved: /data/crime_8.0.geojson
Saved: /data/crime_2.0.geojson
Saved: /data/crime_129.0.geojson
Saved: /data/crime_60.0.geojson
Saved: /data/crime_153.0.geojson
Saved: /data/crime_16.0.geojson
Saved: /data/crime_168.0.geojson
Saved: /data/crime_130.0.geojson
Saved: /data/crime_41.0.geojson
Saved: /data/crime_142.0.geojson
Saved: /data/crime_146.0.geojson
Saved: /data/crime_67.0.geojson
Saved: /data/crime_98.0.geojson
Saved: /data/crime_13.0.geojson
Saved: /data/crime_157.0.geojson
Saved: /data/crime_162.0.geojson
Saved: /data/crime_116.0.geojson
Saved: /data/crime_149.0.geojson
Saved: /data/crime_102.0.geojson
Saved: /data/crime_86.0.geojson
Saved: /data/crime_133.0.geojson
Saved: /data/crime_101.0.geo

In [43]:
###Create Annual Population Fields for gdf_nbr
rate_base = 100000  # Standard rate base for crime rates

# Loop through each year to calculate population based on ASSAULT and ASSAULT_RATE
for year in range(2014, 2023):
    total_incidents_column = f'ASSAULT_{year}'
    rate_column = f'ASSAULT_RATE_{year}'
    population_column = f'POPULATION_{year}'

    # Calculate population for each year
    # Ensure to handle division by zero or NaN values in rates
    gdf_nbr[population_column] = (gdf_nbr[total_incidents_column] / (gdf_nbr[rate_column] / rate_base)).fillna(0).replace([np.inf, -np.inf], 0)

print(gdf_nbr.columns)

Index(['geometry', 'OBJECTID_1', 'AREA_NAME', 'HOOD_ID', 'ASSAULT_2014',
       'ASSAULT_2015', 'ASSAULT_2016', 'ASSAULT_2017', 'ASSAULT_2018',
       'ASSAULT_2019',
       ...
       'Shape__Length', 'POPULATION_2014', 'POPULATION_2015',
       'POPULATION_2016', 'POPULATION_2017', 'POPULATION_2018',
       'POPULATION_2019', 'POPULATION_2020', 'POPULATION_2021',
       'POPULATION_2022'],
      dtype='object', length=196)


In [116]:
print(gdf_nbr.head(10))

   OBJECTID_1                  AREA_NAME HOOD_ID  ASSAULT_2014  ASSAULT_2015   
0           1  South Eglinton-Davisville     174            63            61  \
1           2              North Toronto     173            45            52   
2           3         Dovercourt Village     172            56            57   
3           4   Junction-Wallace Emerson     171           154           157   
4           5         Yonge-Bay Corridor     170           394           524   
5           6             Bay-Cloverhill     169           104           100   
6           7        Bendale-Glen Andrew     156           137           148   
7           8                  Downsview     155           114           127   
8           9   Oakdale-Beverley Heights     154           221           232   
9          10                   Avondale     153            38            40   

   ASSAULT_2016  ASSAULT_2017  ASSAULT_2018  ASSAULT_2019  ASSAULT_2020   
0            70            82            85 

In [86]:
##Create a new gdf_nbr with the geometry and summaries of the incidents by CRIME and OCC_YEAR
merged_gdf['OCC_YEAR'] = merged_gdf['OCC_YEAR'].astype('Int64')
merged_gdf['HOOD_ID'] = merged_gdf['HOOD_ID'].astype(str)

# Filter out rows where OCC_YEAR is NaN in addition to being less than 2014
filtered_gdf = merged_gdf[pd.notna(merged_gdf['OCC_YEAR']) & (merged_gdf['OCC_YEAR'] >= 2014)]


# Group by HOOD_ID, OCC_YEAR, and CRIME, then sum the Incidents
filtered_gdf['Incidents'] = 1
grouped = filtered_gdf.groupby(['HOOD_ID', 'OCC_YEAR', 'CRIME'])['Incidents'].sum().reset_index()

# Initial pivot operation
pivot_table = grouped.pivot_table(index='HOOD_ID', 
                                  columns=['OCC_YEAR', 'CRIME'], 
                                  values='Incidents', 
                                  fill_value=0)

# Create new column names by concatenating CRIME and OCC_YEAR
pivot_table.columns = [crime + '_' + str(year) for year, crime in pivot_table.columns]

# Reset the index to turn HOOD_ID back into a column
pivot_table.reset_index(inplace=True)

# Select only 'HOOD_ID' and 'geometry' from gdf_nbr for the merge
gdf_nbr_selected = gdf_nbr[['HOOD_ID', 'geometry','POPULATION_2014', 'POPULATION_2015', 'POPULATION_2016',
       'POPULATION_2017', 'POPULATION_2018', 'POPULATION_2019',
       'POPULATION_2020', 'POPULATION_2021', 'POPULATION_2022','POPULATION_2023']]


# Ensure data types match for the merging key
gdf_nbr_selected['HOOD_ID'] = gdf_nbr_selected['HOOD_ID'].astype(str)
pivot_table['HOOD_ID'] = pivot_table['HOOD_ID'].astype(str).str.replace('.0', '', regex=False)


# Strip whitespaces from HOOD_ID
gdf_nbr_selected['HOOD_ID'] = gdf_nbr_selected['HOOD_ID'].str.strip()
pivot_table['HOOD_ID'] = pivot_table['HOOD_ID'].str.strip()

# Print unique HOOD_ID values
print(sorted(gdf_nbr_selected['HOOD_ID'].unique()))
print(sorted(pivot_table['HOOD_ID'].unique()))

# Perform the merge
#gdf_nbr_new = gdf_nbr_selected.merge(pivot_table, on='HOOD_ID', how='left')
gdf_nbr_new = pivot_table.merge(gdf_nbr_selected, on='HOOD_ID', how='left')


print(gdf_nbr_new.columns)

/Users/tomweatherburn/anaconda3/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


['1', '10', '100', '101', '102', '103', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '128', '129', '13', '130', '133', '134', '135', '136', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '18', '19', '2', '20', '21', '22', '23', '24', '25', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '46', '47', '48', '49', '5', '50', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '78', '79', '8', '80', '81', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '94', '95', '96', '97', '98

/Users/tomweatherburn/anaconda3/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/tomweatherburn/anaconda3/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [93]:
for year in range(2014, 2023):
    major_crimes = (gdf_nbr_new[f'Assault_{year}'] + gdf_nbr_new[f'Auto Theft_{year}'] +
                    gdf_nbr_new[f'Break and Enter_{year}'] + gdf_nbr_new[f'Homicide_{year}'] +
                    gdf_nbr_new[f'Robbery_{year}'] + gdf_nbr_new[f'Shooting (Non-Fatal)_{year}'] +
                    gdf_nbr_new[f'Theft Over_{year}'])
    
    minor_crimes = gdf_nbr_new[f'Theft From Motor Vehicle_{year}'] + gdf_nbr_new[f'Bike Theft_{year}']
    
    total_crimes = major_crimes + minor_crimes
    
    population = gdf_nbr_new[f'POPULATION_{year}']
    
    gdf_nbr_new[f'MAJOR_CRIME_RATE_{year}'] = (major_crimes / population) * 100000
    gdf_nbr_new[f'MINOR_CRIME_RATE_{year}'] = (minor_crimes / population) * 100000
    gdf_nbr_new[f'TOTALCRIME_RATE_{year}'] = (total_crimes / population) * 100000


In [100]:
print(gdf_nbr_new[['MAJOR_CRIME_RATE_2014', 'MINOR_CRIME_RATE_2014', 'TOTALCRIME_RATE_2014']].head())  # Example for 2014


   MAJOR_CRIME_RATE_2014  MINOR_CRIME_RATE_2014  TOTALCRIME_RATE_2014
0            2447.948341            1042.483338           3490.431679
1             725.017452             794.898652           1519.916104
2             594.833455             237.933382            832.766837
3             677.996731             262.158736            940.155467
4             678.488815             370.084808           1048.573624


In [105]:
# Convert the DataFrame back to a GeoDataFrame, specifying the geometry column
gdf_nbr_new = gpd.GeoDataFrame(gdf_nbr_new, geometry='geometry')

#gdf_nbr_new.to_csv(dir+"/data/gdf_nbr_new.csv", index=False)

# Use the to_file method to write the GeoDataFrame to a GeoJSON file
gdf_nbr_new.to_file(dir+"/data/gdf_nbr_new.geojson", driver='GeoJSON')

print(f"GeoJSON file saved")

GeoJSON file saved


In [103]:
# Ensure you're using the correct GeoDataFrame, assuming it's `gdf_nbr_new`
nbr_geojson = gdf_nbr_new.to_json()

# Proceed with creating the folium map
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=11.5)

# Since you're using 'ASSAULT_RATE_2022', make sure 'gdf_nbr_new' has 'HOOD_ID' and 'ASSAULT_RATE_2022' columns correctly set up
folium.Choropleth(
    geo_data=nbr_geojson,  # GeoJSON data
    name="choropleth",
    data=gdf_nbr_new,  # DataFrame
    columns=['HOOD_ID', 'ASSAULT_RATE_2022'],  # Columns for HOOD_ID and the rate
    key_on='feature.properties.HOOD_ID',  # The path to HOOD_ID in GeoJSON
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Assault Rate in Toronto by Neighborhood (2022)'
).add_to(map_toronto)

# To display in Jupyter Notebook
map_toronto


KeyError: 'ASSAULT_RATE_2022'